# NLP - Session 11 - Real Time Twitter Sentiment Analysis

Need access to Twitter API.

https://developer.twitter.com/app/new

Install tweepy

`!pip install tweepy` or `conda install --name tensorflow20 -c conda-forge tweepy`

In [ ]:
import tweepy

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(auth)

## Get my timeline tweets

In [ ]:
tweets = api.home_timeline()

In [ ]:
tweets

In [ ]:
for tweet in tweets:
    print(tweet.text)

## Get tweet stream and do sentiment analysis

In [ ]:
import json

from textblob import TextBlob
from tweepy import Stream, StreamListener

In [ ]:
import csv
import re

## Create Listener

In [ ]:
trump = 0
biden = 0

header_name = ["Trump", "Biden"]

with open("data\tweeter-real_time-sentiment.csv", "w") as file:
    writer = csv.DictWriter(file, fieldnames=header_name)
    writer.writeheader()


class Listener(StreamListener):
    def on_data(self, data):
        raw_tweets = json.loads(data)
        try:
            tweets = raw_tweets["text"]

            # Removing Emojis, Links and User Tags
            tweets = " ".join(
                re.sub(
                    "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\s+)", " ", tweets
                ).split()
            )
            tweets = " ".join(re.sub("RT", " ", tweets).split())

            blob = TextBlob(tweets.strip())

            global trump
            global biden

            trump_sentiment = 0
            biden_sentiment = 0

            for sent in blob.sentences:
                if "Trump" in sent and "Biden" not in sent:
                    trump_sentiment = trump_sentiment + sent.sentiment.polarity
                else:
                    biden_sentiment = biden_sentiment + sent.sentiment.polarity

            trump = trump + trump_sentiment
            biden = biden + biden_sentiment

            with open("data\tweeter-real_time-sentiment.csv", "a") as file:
                writer = csv.DictWriter(file, fieldnames=header_name)
                now = datetime.now()
                info = {"Time": now, "Trump": trump, "Biden": biden}

                writer.writerow(info)

            print(tweets)
            print()
        except:
            print("Error! Run again!")

    def on_error(self, status):
        print(status)

In [ ]:
twitter_stream = Stream(auth, Listener())
twitter_stream.filter(track=["Trump", "Bidden"])

## Plot Graphs

Ploting the graph needs to be a separate python job as the above process is contineous it will never execute this. So if running take the below plot code and run it as a seperate job where it will continue reading the above csv and keep updating the graph

In [ ]:
from itertools import count
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
%matplotlib notebook # Makes sure real time graph is plotted on Jupyter notebook.

In [ ]:
plt.style.use("fivethirtyeight")

In [ ]:
frame_len = 10000

fig = plt.figure(figsize=(9, 6))


def animate(i):
    data = pd.read_csv("data\tweeter-real_time-sentiment.csv")
    x = data["Time"]
    y1 = data["Trump"]
    y2 = data["Biden"]

    if len(y1) <= frame_len:
        plt.cla()
        plt.plot(y1, label="Donald J Trump")
        plt.plot(y2, label="Joe R Biden")
    else:
        plt.cla()
        plt.plot(y1[-frame_len:], label="Donald J Trump")
        plt.plot(y2[-frame_len:], label="Joe R Biden")

    plt.legend(loc="upper left")
    plt.tight_layout()


ani = FuncAnimation(plt.gcf(), animate, interval=1000)